In [81]:
import re
import pandas as pd
import string
import swifter
from nltk.tag import pos_tag
pd.set_option('display.max_colwidth', None)

In [66]:
df = pd.read_csv('Sarcasm on Reddit dataset/train-balanced-sarcasm.csv')

In [67]:
df

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well."
1,0,You do know west teams play against west teams more than east teams right?,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.
2,0,"They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york nigga"" ones are.",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT
...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the technology to create a pig/bird/human hybrid virus.,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen, but there are reports that this virus has bits of human, pig and bird genetic material, which makes it uniquely interesting"
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,"In a move typical of their recent do-nothing approach to CO2 emissions, Canada appoints a tar sands executive to liaise with the US on climate change"
1010823,1,Perhaps this is an atheist conspiracy to make Christians look bad?,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church on Time
1010824,1,The Slavs got their own country - it is called Kosovo,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,"I've always been unsettled by that. I hear a lot of Jewish people say that Israel can do whatever it pleases because of the holocaust, but the Slavs, and other races the Germans attempted to annihilate go unnoticed in history?"


In [68]:
df[df['comment'].isna() == 1]
# there are some null value comments that we should drop

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
602,0,NaN,Tornado_Hunter24,Infinitewarfare,3,-1,-1,2016-11,2016-11-24 01:39:02,Any recommendations?
56269,1,NaN,SodokuIRL,leagueoflegends,1,1,0,2016-09,2016-09-28 08:23:04,"LPL shitshow, EU LCS shitshow. What isn't a shitshow then?"
68590,1,NaN,TossTheDog,westchesterpa,1,-1,-1,2016-10,2016-10-11 20:25:52,Car fires smell delicious to you? You should probably get checked out.
135348,0,NaN,So12rovv,R6STeams,1,-1,-1,2016-12,2016-12-22 17:24:05,Will do. EU or NA?
199910,0,NaN,Khvostov_7g-02,tf2,1,1,0,2016-05,2016-05-16 05:46:43,"woah, thanks."
258718,1,NaN,PeaceLovePositivity,vaporents,5,5,0,2016-06,2016-06-22 20:37:11,"No, doing drugs while forming a fetus (your kid's body) is very safe and recommended."
284331,0,NaN,FieldDay,ar15,1,1,0,2016-05,2016-05-20 22:18:51,"Fuck you too, buddy."
312969,0,NaN,dont_wear_a_C,nameaserver,1,1,0,2016-08,2016-08-05 07:36:37,Thu 2016-08-04 accessing vault............unlocked calculating name matrix.........locked in attending gala.....galavanted finding server to name........done initializing TCPoB (TCP over Butler).....initialized establishing secure connection.......secured sending name assignment.......done wait for response.........success. one of our servers renamed to *Riot-In-Janeiro*
328775,1,NaN,wedontlikespaces,todayilearned,6,6,0,2016-08,2016-08-18 12:36:21,What symbol denotes suck?
331735,1,NaN,atlacatl,soccer,1,1,0,2016-06,2016-06-14 13:38:04,Doesn't matter if that experience is garbage


In [69]:
df.dropna(subset=['comment'], inplace=True)

In [70]:
def remove_punctuation(text):
#     no_apostrophe = string.punctuation.replace("'", "")
    pattern = f"[{re.escape(string.punctuation)}]"
    return re.sub(pattern, '', text)

In [71]:
df['no punctuation'] = df['comment'].apply(lambda comment: remove_punctuation(comment))

In [72]:
df

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,no punctuation
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well.",NC and NH
1,0,You do know west teams play against west teams more than east teams right?,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.,You do know west teams play against west teams more than east teams right
2,0,"They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.,They were underdogs earlier today but since Gronks announcement this afternoon the Vegas line has moved to patriots 1
3,0,"This meme isn't funny none of the ""new york nigga"" ones are.",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz,This meme isnt funny none of the new york nigga ones are
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT,I could use one of those tools
...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the technology to create a pig/bird/human hybrid virus.,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen, but there are reports that this virus has bits of human, pig and bird genetic material, which makes it uniquely interesting",Im sure that Iran and N Korea have the technology to create a pigbirdhuman hybrid virus
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,"In a move typical of their recent do-nothing approach to CO2 emissions, Canada appoints a tar sands executive to liaise with the US on climate change",whatever you do dont vote green
1010823,1,Perhaps this is an atheist conspiracy to make Christians look bad?,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church on Time,Perhaps this is an atheist conspiracy to make Christians look bad
1010824,1,The Slavs got their own country - it is called Kosovo,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,"I've always been unsettled by that. I hear a lot of Jewish people say that Israel can do whatever it pleases because of the holocaust, but the Slavs, and other races the Germans attempted to annihilate go unnoticed in history?",The Slavs got their own country it is called Kosovo


In [73]:
# now we tokenize/split
from nltk.tokenize import word_tokenize
df['tokenized'] = df['no punctuation'].apply(lambda comment: word_tokenize(comment))

In [74]:
df

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,no punctuation,tokenized
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well.",NC and NH,"[NC, and, NH]"
1,0,You do know west teams play against west teams more than east teams right?,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.,You do know west teams play against west teams more than east teams right,"[You, do, know, west, teams, play, against, west, teams, more, than, east, teams, right]"
2,0,"They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.,They were underdogs earlier today but since Gronks announcement this afternoon the Vegas line has moved to patriots 1,"[They, were, underdogs, earlier, today, but, since, Gronks, announcement, this, afternoon, the, Vegas, line, has, moved, to, patriots, 1]"
3,0,"This meme isn't funny none of the ""new york nigga"" ones are.",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz,This meme isnt funny none of the new york nigga ones are,"[This, meme, isnt, funny, none, of, the, new, york, nigga, ones, are]"
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT,I could use one of those tools,"[I, could, use, one, of, those, tools]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the technology to create a pig/bird/human hybrid virus.,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen, but there are reports that this virus has bits of human, pig and bird genetic material, which makes it uniquely interesting",Im sure that Iran and N Korea have the technology to create a pigbirdhuman hybrid virus,"[Im, sure, that, Iran, and, N, Korea, have, the, technology, to, create, a, pigbirdhuman, hybrid, virus]"
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,"In a move typical of their recent do-nothing approach to CO2 emissions, Canada appoints a tar sands executive to liaise with the US on climate change",whatever you do dont vote green,"[whatever, you, do, dont, vote, green]"
1010823,1,Perhaps this is an atheist conspiracy to make Christians look bad?,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church on Time,Perhaps this is an atheist conspiracy to make Christians look bad,"[Perhaps, this, is, an, atheist, conspiracy, to, make, Christians, look, bad]"
1010824,1,The Slavs got their own country - it is called Kosovo,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,"I've always been unsettled by that. I hear a lot of Jewish people say that Israel can do whatever it pleases because of the holocaust, but the Slavs, and other races the Germans attempted to annihilate go unnoticed in history?",The Slavs got their own country it is called Kosovo,"[The, Slavs, got, their, own, country, it, is, called, Kosovo]"


In [75]:
def is_sticky_caps(text):
    # Filter out non-alphabetic characters
    filtered_chars = [char for char in text if char.isalpha()]

    # If there are fewer than 2 letters, we can't evaluate sticky caps
    if len(filtered_chars) < 2:
        return False

    # Check for sticky caps alternation
    for i in range(1, len(filtered_chars)):
        if filtered_chars[i].isupper() == filtered_chars[i - 1].isupper():
            return False  # Two consecutive letters have the same case

    return True

In [76]:
df['sticky caps'] = df['no punctuation'].apply(lambda comment: is_sticky_caps(comment))

In [77]:
df

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,no punctuation,tokenized,sticky caps
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well.",NC and NH,"[NC, and, NH]",False
1,0,You do know west teams play against west teams more than east teams right?,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.,You do know west teams play against west teams more than east teams right,"[You, do, know, west, teams, play, against, west, teams, more, than, east, teams, right]",False
2,0,"They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.,They were underdogs earlier today but since Gronks announcement this afternoon the Vegas line has moved to patriots 1,"[They, were, underdogs, earlier, today, but, since, Gronks, announcement, this, afternoon, the, Vegas, line, has, moved, to, patriots, 1]",False
3,0,"This meme isn't funny none of the ""new york nigga"" ones are.",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz,This meme isnt funny none of the new york nigga ones are,"[This, meme, isnt, funny, none, of, the, new, york, nigga, ones, are]",False
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT,I could use one of those tools,"[I, could, use, one, of, those, tools]",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the technology to create a pig/bird/human hybrid virus.,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen, but there are reports that this virus has bits of human, pig and bird genetic material, which makes it uniquely interesting",Im sure that Iran and N Korea have the technology to create a pigbirdhuman hybrid virus,"[Im, sure, that, Iran, and, N, Korea, have, the, technology, to, create, a, pigbirdhuman, hybrid, virus]",False
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,"In a move typical of their recent do-nothing approach to CO2 emissions, Canada appoints a tar sands executive to liaise with the US on climate change",whatever you do dont vote green,"[whatever, you, do, dont, vote, green]",False
1010823,1,Perhaps this is an atheist conspiracy to make Christians look bad?,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church on Time,Perhaps this is an atheist conspiracy to make Christians look bad,"[Perhaps, this, is, an, atheist, conspiracy, to, make, Christians, look, bad]",False
1010824,1,The Slavs got their own country - it is called Kosovo,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,"I've always been unsettled by that. I hear a lot of Jewish people say that Israel can do whatever it pleases because of the holocaust, but the Slavs, and other races the Germans attempted to annihilate go unnoticed in history?",The Slavs got their own country it is called Kosovo,"[The, Slavs, got, their, own, country, it, is, called, Kosovo]",False


In [82]:
df['pos'] = df['tokenized'].swifter.apply(pos_tag)

Pandas Apply: 100%|████████████████████████████████████████████████████████| 1010771/1010771 [10:45<00:00, 1565.81it/s]


In [83]:
df

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,no punctuation,tokenized,sticky caps,pos
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well.",NC and NH,"[NC, and, NH]",False,"[(NC, NNP), (and, CC), (NH, NNP)]"
1,0,You do know west teams play against west teams more than east teams right?,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.,You do know west teams play against west teams more than east teams right,"[You, do, know, west, teams, play, against, west, teams, more, than, east, teams, right]",False,"[(You, PRP), (do, VBP), (know, VB), (west, JJS), (teams, NNS), (play, VBP), (against, IN), (west, JJ), (teams, NNS), (more, RBR), (than, IN), (east, JJ), (teams, NNS), (right, RB)]"
2,0,"They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.,They were underdogs earlier today but since Gronks announcement this afternoon the Vegas line has moved to patriots 1,"[They, were, underdogs, earlier, today, but, since, Gronks, announcement, this, afternoon, the, Vegas, line, has, moved, to, patriots, 1]",False,"[(They, PRP), (were, VBD), (underdogs, NNS), (earlier, RBR), (today, NN), (but, CC), (since, IN), (Gronks, NNP), (announcement, NN), (this, DT), (afternoon, NN), (the, DT), (Vegas, NNP), (line, NN), (has, VBZ), (moved, VBN), (to, TO), (patriots, VB), (1, CD)]"
3,0,"This meme isn't funny none of the ""new york nigga"" ones are.",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz,This meme isnt funny none of the new york nigga ones are,"[This, meme, isnt, funny, none, of, the, new, york, nigga, ones, are]",False,"[(This, DT), (meme, NN), (isnt, VB), (funny, JJ), (none, NN), (of, IN), (the, DT), (new, JJ), (york, NN), (nigga, JJ), (ones, NNS), (are, VBP)]"
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT,I could use one of those tools,"[I, could, use, one, of, those, tools]",False,"[(I, PRP), (could, MD), (use, VB), (one, CD), (of, IN), (those, DT), (tools, NNS)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the technology to create a pig/bird/human hybrid virus.,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen, but there are reports that this virus has bits of human, pig and bird genetic material, which makes it uniquely interesting",Im sure that Iran and N Korea have the technology to create a pigbirdhuman hybrid virus,"[Im, sure, that, Iran, and, N, Korea, have, the, technology, to, create, a, pigbirdhuman, hybrid, virus]",False,"[(Im, NNP), (sure, NN), (that, WDT), (Iran, NNP), (and, CC), (N, NNP), (Korea, NNP), (have, VBP), (the, DT), (technology, NN), (to, TO), (create, VB), (a, DT), (pigbirdhuman, JJ), (hybrid, JJ), (virus, NN)]"
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,"In a move typical of their recent do-nothing approach to CO2 emissions, Canada appoints a tar sands executive to liaise with the US on climate change",whatever you do dont vote green,"[whatever, you, do, dont, vote, green]",False,"[(whatever, WDT), (you, PRP), (do, VBP), (dont, JJ), (vote, NN), (green, JJ)]"
1010823,1,Perhaps this is an atheist conspiracy to make Christians look bad?,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church on Time,Perhaps this is an atheist conspiracy to make Christians look bad,"[Perhaps, this, is, an, atheist, conspiracy, to, make, C

In [84]:
df.to_csv('tokenized_tagged.csv')

In [85]:
token_tag = pd.read_csv('tokenized_tagged.csv')

In [86]:
token_tag

,Unnamed: 0,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,no punctuation,tokenized,sticky caps,pos
0,0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well.",NC and NH,"['NC', 'and', 'NH']",False,"[('NC', 'NNP'), ('and', 'CC'), ('NH', 'NNP')]"
1,1,0,You do know west teams play against west teams more than east teams right?,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.,You do know west teams play against west teams more than east teams right,"['You', 'do', 'know', 'west', 'teams', 'play', 'against', 'west', 'teams', 'more', 'than', 'east', 'teams', 'right']",False,"[('You', 'PRP'), ('do', 'VBP'), ('know', 'VB'), ('west', 'JJS'), ('teams', 'NNS'), ('play', 'VBP'), ('against', 'IN'), ('west', 'JJ'), ('teams', 'NNS'), ('more', 'RBR'), ('than', 'IN'), ('east', 'JJ'), ('teams', 'NNS'), ('right', 'RB')]"
2,2,0,"They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.,They were underdogs earlier today but since Gronks announcement this afternoon the Vegas line has moved to patriots 1,"['They', 'were', 'underdogs', 'earlier', 'today', 'but', 'since', 'Gronks', 'announcement', 'this', 'afternoon', 'the', 'Vegas', 'line', 'has', 'moved', 'to', 'patriots', '1']",False,"[('They', 'PRP'), ('were', 'VBD'), ('underdogs', 'NNS'), ('earlier', 'RBR'), ('today', 'NN'), ('but', 'CC'), ('since', 'IN'), ('Gronks', 'NNP'), ('announcement', 'NN'), ('this', 'DT'), ('afternoon', 'NN'), ('the', 'DT'), ('Vegas', 'NNP'), ('line', 'NN'), ('has', 'VBZ'), ('moved', 'VBN'), ('to', 'TO'), ('patriots', 'VB'), ('1', 'CD')]"
3,3,0,"This meme isn't funny none of the ""new york nigga"" ones are.",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz,This meme isnt funny none of the new york nigga ones are,"['This', 'meme', 'isnt', 'funny', 'none', 'of', 'the', 'new', 'york', 'nigga', 'ones', 'are']",False,"[('This', 'DT'), ('meme', 'NN'), ('isnt', 'VB'), ('funny', 'JJ'), ('none', 'NN'), ('of', 'IN'), ('the', 'DT'), ('new', 'JJ'), ('york', 'NN'), ('nigga', 'JJ'), ('ones', 'NNS'), ('are', 'VBP')]"
4,4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT,I could use one of those tools,"['I', 'could', 'use', 'one', 'of', 'those', 'tools']",False,"[('I', 'PRP'), ('could', 'MD'), ('use', 'VB'), ('one', 'CD'), ('of', 'IN'), ('those', 'DT'), ('tools', 'NNS')]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010766,1010821,1,I'm sure that Iran and N. Korea have the technology to create a pig/bird/human hybrid virus.,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen, but there are reports that this virus has bits of human, pig and bird genetic material, which makes it uniquely interesting",Im sure that Iran and N Korea have the technology to create a pigbirdhuman hybrid virus,"['Im', 'sure', 'that', 'Iran', 'and', 'N', 'Korea', 'have', 'the', 'technology', 'to', 'create', 'a', 'pigbirdhuman', 'hybrid', 'virus']",False,"[('Im', 'NNP'), ('sure', 'NN'), ('that', 'WDT'), ('Iran', 'NNP'), ('and', 'CC'), ('N', 'NNP'), ('Korea', 'NNP'), ('have', 'VBP'), ('the', 'DT'), ('technology', 'NN'), ('to', 'TO'), ('create', 'VB'), ('a', 'DT'), ('pigbirdhuman', 'JJ'), ('hybrid', 'JJ'), ('virus', 'NN')]"
1010767,1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,"In a move typical of their recent do-nothing approach to CO2 emissions, Canada appoints a tar sands executive to liaise with the US on climate change",wha